In [ ]:
from torch_geometric.datasets import OPFDataset
import pandas as pd
import numpy as np

### OPF Data

In [2]:
train_np = OPFDataset(
    root="/Users/tamaragovindasamy/Desktop/gridfm-datakit/data/OPF/train/",
    split="train",
    case_name="pglib_opf_case118_ieee",
    num_groups=1,
    topological_perturbations=False,
)

In [6]:
bus_numbers = []
bus_types = []
va_arr = []
vm_arr = []
scenarios = []
PV = []
PQ = []
REF = []
pg_arr = []
qg_arr = []
pd_arr = []
qd_arr = []

count = 0
for graph in train_np[:]:
    n_buses = graph["bus"].y.shape[0]
    for bus in range(n_buses):
        va_val = np.rad2deg(graph["bus"].y[bus, 0].item())
        vm_val = graph["bus"].y[bus, 1].item()
        bus_val = bus
        bus_type_val = graph["bus"].x[bus, 1].item()
        bus_numbers.append(bus)
        bus_types.append(bus_type_val)
        va_arr.append(va_val)
        vm_arr.append(vm_val)
        scenarios.append(count)
        # if bus is PV or PQ, append to the respective lists
        if bus_type_val == 1.0:
            PV.append(0)
            REF.append(0)
            PQ.append(1)
        elif bus_type_val == 2.0:
            PV.append(1)
            REF.append(0)
            PQ.append(0)
        elif bus_type_val == 3.0:
            PV.append(0)
            REF.append(1)
            PQ.append(0)
        # generator
        if bus in graph["bus", "generator_link", "generator"]["edge_index"][0]:
            # if bus is connected to a generator, append the generator values
            pg_val = (
                graph["generator"]
                .y[
                    graph["generator", "generator_link", "bus"]["edge_index"][1] == bus,
                    0,
                ]
                .item()
                * graph.x.item()
            )  # baseMVA
            qg_val = (
                graph["generator"]
                .y[
                    graph["generator", "generator_link", "bus"]["edge_index"][1] == bus,
                    1,
                ]
                .item()
                * graph.x.item()
            )  # baseMVA
            pg_arr.append(pg_val)
            qg_arr.append(qg_val)
        else:
            # if bus is not connected to a generator, append 0
            pg_arr.append(0)
            qg_arr.append(0)
        # load
        if bus in graph["bus", "load_link", "load"].edge_index[0]:
            # if bus is connected to a load, append the load values
            pd_val = (
                graph["load"]
                .x[graph["load", "load_link", "bus"]["edge_index"][1] == bus, 0]
                .item()
                * graph.x.item()
            )  # baseMVA
            qd_val = (
                graph["load"]
                .x[graph["load", "load_link", "bus"]["edge_index"][1] == bus, 1]
                .item()
                * graph.x.item()
            )  # baseMVA
            pd_arr.append(pd_val)
            qd_arr.append(qd_val)
        else:
            # if bus is not connected to a load, append 0
            pd_arr.append(0)
            qd_arr.append(0)
    count += 1

In [7]:
data_for_df = {
    "scenario": scenarios,
    "bus": bus_numbers,
    "Pd": pd_arr,
    "Qd": qd_arr,
    "Pg": pg_arr,
    "Qg": qg_arr,
    "Vm": vm_arr,
    "Va": va_arr,
    "PQ": PQ,
    "PV": PV,
    "REF": REF,
    "bus_type": bus_types,
}
df = pd.DataFrame(data_for_df)  # create df
df.to_csv(
    "/Users/tamaragovindasamy/Desktop/gridfm-datakit/data/OPF/train/dataset_release_1/pglib_opf_case118_ieee/processed_1/train_no_pert.csv",
    index=False,
)  # save to file

In [ ]:
df

### PgLearn Data

In [1]:
import h5py
import pandas as pd
import numpy as np
import json

In [2]:
# Input dataset for Pd, Qd: (N, L)
input_data = "/Users/tamaragovindasamy/Desktop/gridfm-datakit/data/pglearn/input.h5"
input_dataset = h5py.File(input_data, "r")

# Primal dataset for Pg, Qg: (N, G), Va, Vm: (N, N)
primal_data = "/Users/tamaragovindasamy/Desktop/gridfm-datakit/data/pglearn/primal.h5"
primal_dataset = h5py.File(primal_data, "r")

# Dual dataset for Slack_bus: (N, 1)
dual_data = "/Users/tamaragovindasamy/Desktop/gridfm-datakit/data/pglearn/dual.h5"
dual_dataset = h5py.File(dual_data, "r")

# Metadata
meta_data = "/Users/tamaragovindasamy/Desktop/gridfm-datakit/data/pglearn/meta.h5"
meta_dataset = h5py.File(meta_data, "r")

# Case file
with open(
    "/Users/tamaragovindasamy/Desktop/gridfm-datakit/data/pglearn/case.json", "r"
) as f:
    case_data = json.load(f)

In [ ]:
meta_columns = meta_dataset.keys()
for column in meta_columns:
    print(f"Column: {column}, Shape: {meta_dataset[column].shape}")

In [ ]:
input_columns = input_dataset.keys()
for column in input_columns:
    print(f"Column: {column}, Shape: {input_dataset[column].shape}")

In [ ]:
primal_columns = primal_dataset.keys()
for column in primal_columns:
    print(f"Column: {column}, Shape: {primal_dataset[column].shape}")

In [ ]:
dual_columns = dual_dataset.keys()
for column in dual_columns:
    print(f"Column: {column}, Shape: {dual_dataset[column].shape}")

In [4]:
pg = np.array(primal_dataset["pg"])
qg = np.array(primal_dataset["qg"])
vm = np.array(primal_dataset["vm"])
va = np.array(primal_dataset["va"])
Pd = np.array(input_dataset["pd"])
qd = np.array(input_dataset["qd"])

In [ ]:
bus_numbers = []
va_arr = []
vm_arr = []
scenarios = []
pg_arr = []
qg_arr = []
pd_arr = []
qd_arr = []
PV = []
PQ = []
REF = []

n_scenarios = primal_dataset["vm"].shape[0]
n_buses = primal_dataset["vm"].shape[1]
for scenario in range(n_scenarios):
    for bus in range(n_buses):
        va_val = np.rad2deg(va[scenario, bus])
        vm_val = vm[scenario, bus]
        bus_numbers.append(bus)
        va_arr.append(va_val)
        vm_arr.append(vm_val)
        scenarios.append(scenario)
        # generator
        if not case_data["data"]["bus_gens"][bus]:
            pg_arr.append(0)
            qg_arr.append(0)
        else:
            pg_val = (
                pg[scenario, case_data["data"]["bus_gens"][bus][0] - 1]
                * case_data["data"]["base_mva"]
            )  # indices go up to 54
            qg_val = (
                qg[scenario, case_data["data"]["bus_gens"][bus][0] - 1]
                * case_data["data"]["base_mva"]
            )
            pg_arr.append(pg_val)
            qg_arr.append(qg_val)

        # loads
        if not case_data["data"]["bus_loads"][bus]:
            pd_arr.append(0)
            qd_arr.append(0)
        else:
            pd_val = (
                Pd[scenario, case_data["data"]["bus_loads"][bus][0] - 1]
                * case_data["data"]["base_mva"]
            )  # indices go up to 99
            qd_val = (
                qd[scenario, case_data["data"]["bus_loads"][bus][0] - 1]
                * case_data["data"]["base_mva"]
            )
            pd_arr.append(pd_val)
            qd_arr.append(qd_val)

In [ ]:
data_for_df = {
    "scenario": scenarios,
    "bus": bus_numbers,
    "Pd": pd_arr,
    "Qd": qd_arr,
    "Pg": pg_arr,
    "Qg": qg_arr,
    "Vm": vm_arr,
    "Va": va_arr,
}
df = pd.DataFrame(data_for_df)  # create df

In [ ]:
df

In [8]:
df.to_csv(
    "/Users/tamaragovindasamy/Desktop/gridfm-datakit/data/pglearn/pglearn.csv",
    index=False,
)